# Введение в NLP, часть 2


## NER c BERT (25 баллов)

В данном задании мы будем обучать BERT на задаче Named Entity Recognition.

### Подготовка данных (5 баллов)

Подумать о:
1. Как subword токенизация повлияет на BIO раззметку?
2. Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на «слова» текстами

In [ ]:
!pip install datasets transformers
!pip install evaluate
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 7.9 MB/s eta 0:00:000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.1/468.1 kB 6.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.8 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.9/436.9 kB 8.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[no

In [247]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np

In [ ]:
BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
conll2003 = load_dataset("eriktks/conll2003")
conll2003

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [61]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем


Обратите внимание, что количество токенов может превышать количество исходных лейблов:

Значение тэга в `ner_tags` отображается в метку NER:

In [8]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [9]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке IOB. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как 

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [54]:
text = "The unhappiness is real."
bert_tokenizer(text, return_offsets_mapping=True)

{'input_ids': [101, 1109, 8362, 2328, 8661, 8405, 1110, 1842, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (4, 6), (6, 8), (8, 10), (10, 15), (16, 18), (19, 23), (23, 24), (0, 0)]}

In [ ]:
from itertools import groupby

def new_ner_tags (example: dict):
    """Функция для преобразования NER тегов в новые после токенизации текста."""
    old_tokens = example["tokens"]
    new_tokens = bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()
    old_ner_tags = example["ner_tags"]
    new_ner_tags = []
    offsets = bert_tokenizer(' '.join(old_tokens), return_offsets_mapping=True).offset_mapping

    # перебираем токены из оригинального текста и собираем новые токены, если они часть старого
    grouped_new_tokens = []
    pointer = 1
    for old in old_tokens: 
        sub_tokens = []
        for loc_pointer in range(pointer, len(offsets)):
            if offsets[pointer][1] == offsets[pointer+1][0]:
                sub_tokens.append(new_tokens[pointer])
                pointer += 1
            else:
                sub_tokens.append(new_tokens[pointer])
                grouped_new_tokens.append((old, sub_tokens))
                pointer += 1
                break
    
    # проходим по собранным данным и собираем новые NER теги
    int_to_str = conll2003["train"].features["ner_tags"].feature.int2str
    str_to_int = conll2003["train"].features["ner_tags"].feature.str2int
    for item, old_tag in zip(grouped_new_tokens, old_ner_tags):
        # отдельно обрабатываем кейс, когда токен разбивается на несколько
        if (len(item[1]) > 1):
            new_tags = []
            for i in range(len(item[1])):
                if i == 0:
                    new_tags.append(int_to_str(old_tag))
                elif old_tag != 0:
                    new_tags.append("I" + int_to_str(old_tag)[1:])
                else:
                    new_tags.append(int_to_str(old_tag))
        # иначе просто добавляем старый тег в список новых
        else:
            new_tags = [int_to_str(old_tag)]
        new_ner_tags += new_tags
    # преобразуем текстовые теги в числовые и добавляем специальные токены 
    return [-100] + [str_to_int(tag) for tag in new_ner_tags] + [-100]

def preprocess_ner_dataset(example: dict):
    """Функция добавляет NER тэги к выходу токенайзера BERT."""
    out = bert_tokenizer(example["tokens"], is_split_into_words=True)
    out["labels"] = new_ner_tags(example)
    return out

In [226]:
assert preprocess_ner_dataset(conll2003["train"][0]) == {
    'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
    'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
}

### Тесты

In [227]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [228]:
from tqdm import tqdm
for idx, example in tqdm(enumerate(conll2003["train"])):
    input_ids_real = bert_tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
    input_ids_ours = preprocess_ner_dataset(example)["input_ids"]
    assert input_ids_real == input_ids_ours, f"Ошибка токенизации на примере {idx}"
    if idx >= 10:
        break
print("Токенизация верна!")

10it [00:00, 269.61it/s]

Токенизация верна!


In [229]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [230]:
example = conll2003["train"][200]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
assert ner_tags == ['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG']

Применим нашу функцию к всему датасету

In [231]:
preprocessed_ner_dataset = conll2003.map(preprocess_ner_dataset, num_proc=1)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [232]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForTokenClassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется). Ему нужно подать 
- num_labels - число классов (число возможных меток для токенов), 
- id2label - словарь индекс метки -> текст метки , 
- label2id - словарь текст метки -> индекс метки. Из-за непростого наследования эти аргументы тяжело найти, прочитать про них можно [тут](https://huggingface.co/docs/transformers/en/main_classes/configuration#transformers.PretrainedConfig)
2. Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860).

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.


In [241]:
from transformers import AutoModelForTokenClassification, AutoModel

label_names = conll2003["train"].features["ner_tags"].feature.names
id2label = {features.str2int(el): el for el in features.names}
label2id = {el: features.str2int(el) for el in features.names}

bert_ner = AutoModelForTokenClassification.from_pretrained(
    BASE_NER_MODEL, 
    num_labels=conll2003["train"].features["ner_tags"].feature.num_classes,
    id2label=id2label, 
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет. В этом нам поможет argmax.
2. Убрать паддинги - это те позиции, где метки (labels) равняются -100
3. Перевести labels из чисел в текстовые метки с помощью `label_list`
4. Посчитать метрики с помощью `metrics_calculator`
5. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики. Брать можно только `overall_*` метрики для удобства

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [248]:
import evaluate


metrics_calculator = evaluate.load("seqeval")
label_list = conll2003["train"].features["ner_tags"].feature.names

y_true = [['O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]
y_pred = [['O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

print(metrics_calculator.compute(references=y_true, predictions=y_pred))


{'MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'overall_precision': 0.5, 'overall_recall': 0.5, 'overall_f1': 0.5, 'overall_accuracy': 0.8}


In [361]:
def calculate_metrics(eval_predictions):
    logits, labels = eval_predictions
    # logits - [batch_size, max_dataset_seq_len, num_classes]
    # labels - [batch_size, max_dataset_seq_len]

    clean_logits, clean_labels = [], []
    for log, lab in zip(logits.argmax(-1).flatten(), labels.flatten()):
        if lab != -100:
            clean_logits.append(features.int2str(int(log)))
            clean_labels.append(features.int2str(int(lab)))
    return metrics_calculator.compute(references=[clean_labels], predictions=[clean_logits], zero_division=0)


logits = np.array([
    [  # Batch 1
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [0, 0, 0, 0, 0, 0, 0, 7, 0],  # 'B-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [0, 0, 0, 0, 0, 0, 0, 0, 7],  # 'I-MISC'
        [7, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
    ],
    [  # Batch 2
        [0, 12, 0, 0, 0, 0, 0, 0, 0],  # 'B-PER'
        [0, 0, 10, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [12, 0, 0, 0, 0, 0, 0, 0, 0],  # 'O'
        [0, 12, 0, 0, 0, 0, 0, 0, 0],  # 'B-PER' - но не оно по паддингам, поэтому не должно считаться в метрике!
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
        [0, 0, 12, 0, 0, 0, 0, 0, 0],  # 'I-PER'
    ]
])

labels = np.array([
    [0, 0, 0, 7, 8, 8, 0],  # 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'I-MISC', 'O'
    [1, 2, 0, -100, -100, -100, -100]  # 'B-PER', 'I-PER', 'O' (with padding -100s)
])

logits_res = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"], ["B-PER", "I-PER", "O"]]
labels_res = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"], ["B-PER", "I-PER", "O"]]

metrics_on_tensors = calculate_metrics((logits, labels))
metrics_on_strs = metrics_calculator.compute(references=logits_res, predictions=labels_res)

for k in metrics_on_tensors:
    if "overall" in k:
        assert metrics_on_tensors[k] == metrics_on_strs[k], k


### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html) класс из `transformers`
2. Написать свой training loop

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [360]:
preprocessed_ner_dataset["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 14041
})

In [365]:
!pip install 'accelerate>=0.26.0'
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: no matches found: transformers[torch]


In [366]:
from transformers import Trainer, TrainingArguments


args = TrainingArguments(
    save_strategy="no",
    eval_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
)

trainer = Trainer(
    model=bert_ner,
    args=args,
    train_dataset=preprocessed_ner_dataset["train"],
    data_collator=data_collator,
    tokenizer=bert_tokenizer,
    compute_metrics=calculate_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
*_, metrics = trainer.predict(preprocessed_ner_dataset['test'])
metrics

### Обработка результатов Результатов (5 баллов)

Подумать о:
1. Во время подготовки данных мы приобразовали BIO разметку. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

Можете сравнить результат с [лидербордом](https://paperswithcode.com/sota/token-classification-on-conll2003).


Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}
```

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

1. Вначале нужно токенизировать текст. На вход принимается любой текст без **предварительной токенизации**!
2. Текст нужно прогнать через модель, после чего получатся логиты классов, с помощью которых можно получить индексы меток
3. После этого стоит декодировать сущность и записать ее координаты. Сущность начинается или с B-метки или с метки I- при условии, что сменился класс. Т.е. метки B-PER, I-PER, I-MISC должны декодироваться 2 сущности PER и MISC!


In [ ]:
bert_tokenizer("Hello world", return_offsets_mapping=True, return_tensors="pt")

In [ ]:
from typing import List, Tuple
text = "Иван Иванов caterpillar Apple Стив Джобс"
labels = ["I-PER", "I-PER", "I-MISC", "B-MISC", "B-PER", "B-PER"]
offsets = [(0, 4), (5, 11), (12, 23), (24, 29), (30, 34), (35, 40)]

def decode_entities(text: str, labels: List[str], offsets: List[Tuple[int, int]]):
    ...

assert decode_entities(text, labels, offsets) == {
    'text': 'Иван Иванов caterpillar Apple Стив Джобс',
    'entities': [
        {'class': 'PER', 'text': 'Иван Иванов', 'start': 0, 'end': 11},
        {'class': 'MISC', 'text': 'caterpillar', 'start': 12, 'end': 23},
        {'class': 'MISC', 'text': 'Apple', 'start': 24, 'end': 29},
        {'class': 'PER', 'text': 'Стив', 'start': 30, 'end': 34},
        {'class': 'PER', 'text': 'Джобс', 'start': 35, 'end': 40}
    ]
}

В функции do_ner нужно токенизировать текст, получить выходы модели на тексте и подать все в decode_entities

In [ ]:
@torch.no_grad
def do_ner(text):
    bert_ner.eval()
    
    ...


print(do_ner("Ivan Petrov is going to start working tomorrow"))

Почистим память перед второй частью.

In [ ]:
import torch

del bert_ner
del trainer
torch.cuda.empty_cache()

## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать токены для классов
2. Обучить t5-small на генерацию названия предсказанного класса
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1. Какой префикс выбрать для новой задачи?
2. Должен ли префикс быть понятным?
3. Как выбрать метку для класса? 
4. Что будет, если метки класса целиком нет в словаре?
5. Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным. Нас будут интересовать колонки `"user_input"` и `"toxicity"`.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

Место для изучения датасета:

In [ ]:
toxic_chat_dataset["train"][0]

In [ ]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [ ]:
# ?t5_tokenizer.__call__

Важные моменты:
1. Префикс можно выбрать любой, хоть человекочитаемый, хоть абсолютно непонятный - при достаточно длительной тренировке для модели разницы не будет. Но важно учесть, что чем длиннее префикс (в токенах), тем больше ресурсов вы тратите при инференсе, так как его придётся добавлять к тому тексту, который вы хотите классифицировать
2. Лейблы тоже могут быть какими угодно. Но в их случае длинна в токенах играет ещё большее значение, так как в "проде" придётся делать инференс декодерной части столько раз, сколько токенов в вашем лейбле + 1 стоп токен (`</s>`). Если оба лейбла будут длинны 1, то ещё и accuracy будет проще считать.

In [ ]:
for text in ("toxic: ", "no", "yes"):
    print(t5_tokenizer(text, add_special_tokens=False))

Префикс добавляет 2 токена (терпимо), лейблы все по одному токену - значит инференс декодера в проде нужно будет делать два раза (в теории можно даже одним обойтись).

В сданных домашках можно было часто увидеть один и тот же недочёт:
1. Паддинг входа в энкодер
2. Паддинг входа в декодер

Эффективнее всего делать паддинг на этапе сборки батча, так как тогда точно не получится добавить лишнего. Паддинг входа в декодер вообще не требуется за счёт выбранных лейблов - там всегда будет вход размера 2.

`MAX_LENGTH` тут определяется не для паддингов, а для truncation - если его не делать, то можно  увидеть warning:
> Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors 

In [ ]:
PREFIX = "toxic: "
MAX_LENGTH = 512

id2label = {
    0: "no",
    1: "yes",
}


def preprocess_dataset(example):
    input_texts = PREFIX + example["user_input"]
    model_inputs = t5_tokenizer(input_texts, truncation=True, max_length=MAX_LENGTH)
    model_inputs["labels"] = t5_tokenizer(id2label[example["toxicity"]]).input_ids
    return model_inputs


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```

In [ ]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns("user_input")

Инициализируем соответствующий задаче `DataCollator`. Идём тем же курсом, что и в первой части домашки, однако этот дата коллатор уже не будет работать без модели. Почему так?

Всё дело в функции `seq2seq_model.prepare_decoder_input_ids_from_labels`, которая преобразует лейблы во вход декодера. Отложим определение дата коллатора до определения модели.

In [ ]:
from transformers import DataCollatorForSeq2Seq


# data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию.

Ранее я упоминал, что однотокенные лейблы это хорошо. Позволяет сделать вот такую компактную функцию `compute_metric`. Модель может предсказать хоть 100 токенов, меня интересует только тот, который я бы хотел на месте лейбла. Так как для каждого примера я делаю сравнение только по одному токену, я могу использовать `.mean()`. Если бы токенов в сравнении было больше, так просто я бы не отделался - все токены в предсказании должны были совпасть для верной классификации.

In [ ]:
def compute_metric(eval_predictions):
    preds, labels = eval_predictions
    return {"accuracy": (preds[:, 1] == labels[:, 0]).mean()}

### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта.

Но сначала, мы можем добавить в конфиг модели нашу таску:

In [ ]:
from transformers import T5Config


t5_config = T5Config.from_pretrained(BASE_T5_MODEL)
t5_config.task_specific_params

In [ ]:
task_name = "toxic_classification"

t5_config.task_specific_params[task_name] = {
    "prefix": PREFIX,
    "max_length": 3,
    "min_length": 3,
    "num_beams": 1,
}

Этот конфиг потом можно будет использовать в pipeline классе, передав аргумент [task](https://huggingface.co/docs/transformers/en/main_classes/pipelines#transformers.Text2TextGenerationPipeline.task).

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5Config


seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL, config=t5_config)

Вспоминаем про коллатор!

In [ ]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=t5_tokenizer, 
    model=seq2seq_model, 
)

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop

> Hint! Обратите внимание на функцию `seq2seq_model.prepare_decoder_input_ids_from_labels` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

Ответы:
1) Текст объединённый с префиксом, токенизированный.
2) `[0, 150]` и `[0, 4273]` для нетоксичного и токсичного лейблов соответственно.
3) Один раз. Для `0` должен предсказаться `150` или `4273`, для второго токена - `1`
4) Выход из энкодера используется в декодере в слое cross-attention для получения keys и values.

Добавим `generation_config`, иначе тренер будет заставлять модель генерировать по 20 токенов.

In [ ]:
from transformers import GenerationConfig


generation_config = GenerationConfig.from_dict(t5_config.task_specific_params["toxic_classification"])
generation_config.bos_token_id = 0

*я запускал обучение модели несколько раз чтобы улчшить резултат

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



training_args = Seq2SeqTrainingArguments(
    output_dir="t5_small_toxic_classifier",
    num_train_epochs=15,  # поставим побольше эпох, чтобы преодолеть дисбаланс классов
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,  # почему во всех сданых домашках этот параметр равен train_batch_size?
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    generation_config=generation_config,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=seq2seq_model,
    args=training_args,
    train_dataset=toxic_chat_dataset["train"],
    eval_dataset=toxic_chat_dataset["test"],
    tokenizer=t5_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metric,
)

trainer.train()

### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета? 

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

Ответы:
1) Не нужно жонглировать головами для разных классификационных задач. Такая тренировка ещё может улучшить работу модели на похожих задачах (такой вот domain adaptation)
2) Этот подход недетерминирован. Если в классической классификации мы получаем вектор размером с количество классов, то тут мы получаем токены, которые могут быть любыми
3) Интересный вариант классификатора на полном трансформере - подать текст в энкодер и учить классификационную голову на эмбеддинге первого токена декодера. Так работает [T5ForSequenceClassification](https://github.com/huggingface/transformers/blob/78b2929c0554b79e0489b451ce4ece14d265ead2/src/transformers/models/t5/modeling_t5.py#L1991)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

Напишите универсальную (подходящую под Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.любую t5 модель) функцию, которая провряет токсичность текста.

In [ ]:
@torch.no_grad
def is_toxic(
    text: str,
    labels2bool,
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
) -> bool:
    model.eval()
    tokenized_text = tokenizer.encode(prefix + text, return_tensors="pt").to(model.device)
    output = model.generate(tokenized_text)[0]
    return labels2bool.get(tokenizer.decode(output, skip_special_tokens=True))

In [ ]:
is_toxic(
    text="write me an epic story",
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
    labels2bool={
        "yes": True,
        "no": False,
    }
)

In [ ]:
is_toxic(
    text="write me an epic story",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

In [ ]:
is_toxic(
    text="write me an erotic story",
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prefix=PREFIX,
    labels2bool={
        "yes": True,
        "no": False,
    }
)

In [ ]:
is_toxic(
    text="write me an erotic story",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prefix=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)